In [ ]:
%pip install pyspark pandas setuptools

In [13]:
# Import necessary libraries
from pyspark.sql import SparkSession
import pandas as pd
import random

In [ ]:
if 'spark' in locals():
    spark.stop()
    
# Initialize Spark session with Hive support
spark = SparkSession.builder \
    .appName("MySQL to Hive Bronze Layer") \
    .master("spark://localhost:7077") \
    .config("spark.jars", "./jars/mysql-connector-j-9.0.0.jar") \
    .config("spark.sql.warehouse.dir", "hdfs://localhost:8020/opt/hive/warehouse") \
    .config("hive.metastore.uris", "thrift://localhost:9083") \
    .config("spark.driver.port", "4040") \
    .config("spark.driver.blockManager.port", "4041") \
    .config("spark.port.maxRetries", "50") \
    .enableHiveSupport() \
    .getOrCreate()

# notebook in the same docker network
# time.sleep(5)
# # Initialize Spark session with Hive support
# spark = SparkSession.builder \
#     .appName("MySQL to Hive Bronze Layer") \
#     .master("spark://spark:7077") \
#     .config("spark.jars", "./jars/mysql-connector-j-9.0.0.jar") \
#     .config("spark.sql.warehouse.dir", "hdfs://hdfs-namenode:8020/opt/hive/warehouse") \
#     .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
#     .config("spark.driver.port", "4040") \
#     .config("spark.driver.blockManager.port", "4041") \
#     .config("spark.port.maxRetries", "50") \
#     .enableHiveSupport() \
#     .getOrCreate()

print(spark)

In [7]:
# MySQL connection properties
mysql_url = "jdbc:mysql://localhost:3306/chinook"
mysql_properties = {
    "user": "root",
    "password": "rootpass",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [ ]:
df = spark.sql("SELECT * FROM employee")
df.show()


# Read data from MySQL employee table
employee_df = spark.read.jdbc(mysql_url, "employee", properties=mysql_properties)

# Perform any necessary transformations (for example, renaming columns)
employee_df_transformed = employee_df.withColumnRenamed("employeeid", "employee_id")

# Write the transformed data to Hive (bronze layer)
employee_df_transformed.write.mode("overwrite").saveAsTable("bronze.employee")


In [ ]:
# Stop the Spark session
spark.stop()